Project 3: Classification
Daniel Arthur

Index:
Business Understanding: Notebook clearly explains the project’s value for helping a specific stakeholder solve a real-world problem.
    
    Introduction explains the real-world problem the project aims to solve
    Introduction identifies stakeholders who could use the project and how they would use it
    Conclusion summarizes implications of the project for the real-world problem and stakeholders 


Data Understanding:  Notebook clearly describes the source and properties of the data to show how useful the data are for solving the problem of interest

    Describe the data sources and explain why the data are suitable for the project
    Present the size of the dataset and descriptive statistics for all features used in the analysis
    Justify the inclusion of features based on their properties and relevance for the project
    Identify any limitations of the data that have implications for the project


Data Preparation: Notebook shows how you prepare your data and explains why by including

    Instructions or code needed to get and prepare the raw data for analysis
    Code comments and text to explain what your data preparation code does
    Valid justifications for why the steps you took are appropriate for the problem you are solving


Modeling: Notebook demonstrates an iterative approach to model-building.

    Runs and interprets a simple, baseline model for comparison
    Introduces new models that improve on prior models and interprets their results
    Explicitly justifies model changes based on the results of prior models and the problem context
    Explicitly describes any improvements found from running new models



Evaluation: Notebook shows how well a final model solves the real-world problem.

    Justifies choice of metrics using context of the real-world problem and consequences of errors
    Identifies one final model based on performance on the chosen metrics with validation data
    Evaluates the performance of the final model using holdout test data
    Discusses implications of the final model evaluation for solving the real-world problem


Code Quality: Code in notebook and related files meets professional standards 
    Code is easy to read, using comments, spacing, variable names, and function docstrings
    All code runs and no code or comments are included that are not needed for the project 
    Code minimizes repetition, using loops, functions, and classes
    Code adapted from others is properly cited with author names and location of the cited material


Dummy Model\
Log Regression\
Forests\
    Random Forest\
    Decision Trees\
    Bagged Trees\
Weak Learners\
    Addaboost\
    Gradient Boosting\
KNeighborsClassifier\

Importing the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf


from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.impute import MissingIndicator, SimpleImputer

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score
# plot_confusion_matrix is a handy visual tool, added in the latest version of scikit-learn
# if you are running an older version, comment out this line and just use confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve

Importing the data that will be used.

In [ ]:
data_labels=pd.read_csv('Training_Set_Values.csv')
data_values=pd.read_csv('Training_Set_Labels.csv')
data = data_values.merge(data_labels, on='id')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
# Create dummy variables
#data = pd.get_dummies(salaries)
#data.head()

The target Variable is 'Status_Group'

In [ ]:
data.status_group.value_counts()

In [ ]:
waste_features=['wpt_name','num_private','subvillage','region_code','recorded_by','management_group',
                'extraction_type_group','extraction_type_class','scheme_name','payment','quality_group',
                'quantity_group','source_type','source_class','waterpoint_type_group','ward','installer',
                'public_meeting','permit','date_recorded','construction_year','id']
data.drop(waste_features, axis = 1, inplace = True)

In [ ]:
data.info()

In [ ]:
data['funder'] = pd.factorize(data['funder'])[0]
data['scheme_management'] = pd.factorize(data['scheme_management'])[0]
data['extraction_type'] = pd.factorize(data['extraction_type'])[0]
data['management'] = pd.factorize(data['management'])[0]
data['payment_type'] = pd.factorize(data['payment_type'])[0]
data['water_quality'] = pd.factorize(data['water_quality'])[0]
data['quantity'] = pd.factorize(data['quantity'])[0]
data['source'] = pd.factorize(data['source'])[0]
data['waterpoint_type'] = pd.factorize(data['waterpoint_type'])[0]
data['basin'] = pd.factorize(data['basin'])[0]
data['region'] = pd.factorize(data['region'])[0]
data['lga'] = pd.factorize(data['lga'])[0]
data['district_code'] = pd.factorize(data['district_code'])[0]


In [ ]:
y = data.status_group
X = data.drop('status_group', axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

1st Model - Dummy Model

In [ ]:
dummy_model = DummyClassifier(strategy='stratified')

Fit the model on our data

In [ ]:
dummy_model.fit(X_train, y_train)

In [ ]:
# just grabbing the first 50 to save space
dummy_model.predict(X_train)[:50]

Model Evaluation\
Let's do some cross-validation to see how the model would do in generalizing to new data it's never seen.

In [ ]:
cv_results = cross_val_score(dummy_model, X_train, y_train, cv=5)
cv_results

In [ ]:
confusion_matrix(y_train, dummy_model.predict(X_train))

To show the spread, let's make a convenient class that can help us organize the model and the cross-validation:

In [ ]:
cv_results = cross_val_score(dummy_model, X_train, y_train, cv=5)
cv_results

In [ ]:
class ModelWithCV():
    '''Structure to save the model and more easily see its crossvalidation'''
    
    def __init__(self, model, model_name, X, y, cv_now=True):
        self.model = model
        self.name = model_name
        self.X = X
        self.y = y
        # For CV results
        self.cv_results = None
        self.cv_mean = None
        self.cv_median = None
        self.cv_std = None
        #
        if cv_now:
            self.cross_validate()
        
    def cross_validate(self, X=None, y=None, kfolds=10):
        '''
        Perform cross-validation and return results.
        
        Args: 
          X:
            Optional; Training data to perform CV on. Otherwise use X from object
          y:
            Optional; Training data to perform CV on. Otherwise use y from object
          kfolds:
            Optional; Number of folds for CV (default is 10)  
        '''
        
        cv_X = X if X else self.X
        cv_y = y if y else self.y

        self.cv_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds)
        self.cv_mean = np.mean(self.cv_results)
        self.cv_median = np.median(self.cv_results)
        self.cv_std = np.std(self.cv_results)

        
    def print_cv_summary(self):
        cv_summary = (
        f'''CV Results for `{self.name}` model:
            {self.cv_mean:.5f} ± {self.cv_std:.5f} accuracy
        ''')
        print(cv_summary)

        
    def plot_cv(self, ax):
        '''
        Plot the cross-validation values using the array of results and given 
        Axis for plotting.
        '''
        ax.set_title(f'CV Results for `{self.name}` Model')
        # Thinner violinplot with higher bw
        sns.violinplot(y=self.cv_results, ax=ax, bw=.4)
        sns.swarmplot(
                y=self.cv_results,
                color='orange',
                size=10,
                alpha= 0.8,
                ax=ax
        )

        return ax

In [ ]:
dummy_model_results = ModelWithCV(
                        model=dummy_model,
                        model_name='dummy',
                        X=X_train, 
                        y=y_train)

In [ ]:
fig, ax = plt.subplots()

ax = dummy_model_results.plot_cv(ax)
plt.tight_layout();

dummy_model_results.print_cv_summary()

In [ ]:
fig, ax = plt.subplots()

fig.suptitle("Dummy Model")

plot_confusion_matrix(dummy_model, X_train, y_train, ax=ax, cmap="plasma");

In [ ]:
# just the numbers (this should work even with older scikit-learn)
confusion_matrix(y_train, dummy_model.predict(X_train))

2nd Model Log Regression

In [ ]:
simple_logreg_model = LogisticRegression(random_state=2021, penalty='none')

In [ ]:
simple_logreg_model.fit(X_train, y_train)

In [ ]:
simple_logreg_model.predict(X_train)[:50]

Model Evaluation, Part 2

In [ ]:
simple_logreg_results = ModelWithCV(
                        model=simple_logreg_model,
                        model_name='simple_logreg',
                        X=X_train, 
                        y=y_train
)

In [ ]:
# Saving variable for convenience
model_results = simple_logreg_results

# Plot CV results
fig, ax = plt.subplots()
ax = model_results.plot_cv(ax)
plt.tight_layout();
# Print CV results
model_results.print_cv_summary()

In [ ]:
confusion_matrix(y_train, simple_logreg_model.predict(X_train))

In [ ]:
fig, ax = plt.subplots()

fig.suptitle("Logistic Regression with Numeric Features Only")

plot_confusion_matrix(simple_logreg_model, X_train, y_train, ax=ax, cmap="plasma");

In [ ]:
#plot_roc_curve(simple_logreg_model, X_train, y_train)

Random Forest Classifier

In [ ]:
clf = RandomForestClassifier(n_estimators=500,max_features='auto',
                                         min_samples_split=8)

In [ ]:
clf.fit(X_train, y_train)

Decision Tree Classifier

In [ ]:
# Create the classifier, fit it on the training data and make predictions on the test set
clf = DecisionTreeClassifier(criterion='entropy')

clf.fit(X_train, y_train)

Feature importance

In [ ]:
# Feature importance
tree_clf.feature_importances_

In [ ]:
def plot_feature_importances(model):
    n_features = data_train.shape[1]
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), data_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

plot_feature_importances(tree_clf)

Plot the decision tree

In [ ]:
fig, axes = plt.subplots(nrows = 1,ncols = 1, figsize = (3,3), dpi=300)
tree.plot_tree(clf,
               feature_names = data.columns, 
               class_names=np.unique(y).astype('str'),
               filled = True)
plt.show()

Evlauate the Perfomance of Model

In [ ]:
X_test_ohe = ohe.transform(X_test)
y_preds = clf.predict(X_test_ohe)

print('Accuracy: ', accuracy_score(y_test, y_preds))

Bagged Trees

In [ ]:
# Instantiate a BaggingClassifier
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), 
                                 n_estimators=20)

In [ ]:
# Fit to the training data
bagged_tree.fit(data_train, target_train)

In [ ]:
# Training accuracy score
bagged_tree.score(data_train, target_train)

In [ ]:
# Test accuracy score
bagged_tree.score(data_test, target_test)

Random forests

In [ ]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, max_depth= 5)
forest.fit(data_train, target_train)

In [ ]:
# Training accuracy score
forest.score(data_train, target_train)

In [ ]:
# Test accuracy score
forest.score(data_test, target_test)

In [ ]:
plot_feature_importances(forest)

Look at the trees in your forest

In [ ]:
# Instantiate and fit a RandomForestClassifier
forest_2 = RandomForestClassifier(n_estimators = 5, max_features= 10, max_depth= 2)
forest_2.fit(data_train, target_train)

In [ ]:
# First tree from forest_2
rf_tree_1 = forest_2.estimators_[0]

In [ ]:
# Feature importance
plot_feature_importances(rf_tree_1)

In [ ]:
# Second tree from forest_2
rf_tree_2 = forest_2.estimators_[1]

In [ ]:
# Feature importance
plot_feature_importances(rf_tree_2)

Weak Learners

In [ ]:
# Instantiate an AdaBoostClassifier
adaboost_clf = AdaBoostClassifier(random_state=42)

# Instantiate an GradientBoostingClassifier
gbt_clf = GradientBoostingClassifier(random_state=42)

In [ ]:
# Fit the models
adaboost_clf.fit(X_train, y_train)

gbt_clf.fit(X_train, y_train)

In [ ]:
# AdaBoost model predictions
adaboost_train_preds = adaboost_clf.predict(X_train)
adaboost_test_preds = adaboost_clf.predict(X_test)

# GradientBoosting model predictions
gbt_clf_train_preds = gbt_clf.predict(X_train)
gbt_clf_test_preds = gbt_clf.predict(X_test)

In [ ]:
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds)
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
    
print("Training Metrics")
display_acc_and_f1_score(y_train, adaboost_train_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_train, gbt_clf_train_preds, model_name='Gradient Boosted Trees')
print("")
print("Testing Metrics")
display_acc_and_f1_score(y_test, adaboost_test_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_test, gbt_clf_test_preds, model_name='Gradient Boosted Trees')

In [ ]:
adaboost_confusion_matrix = confusion_matrix(y_test, adaboost_test_preds)
adaboost_confusion_matrix

In [ ]:
gbt_confusion_matrix = confusion_matrix(y_test, gbt_clf_test_preds)
gbt_confusion_matrix

In [ ]:
adaboost_classification_report = classification_report(y_test, adaboost_test_preds)
print(adaboost_classification_report)

In [ ]:
gbt_classification_report = classification_report(y_test, gbt_clf_test_preds)
print(gbt_classification_report)

In [ ]:
print('Mean Adaboost Cross-Val Score (k=5):')
print(cross_val_score(adaboost_clf, df, target, cv=5).mean())

In [ ]:
print('Mean GBT Cross-Val Score (k=5):')
print(cross_val_score(gbt_clf, df, target, cv=5).mean())

KNeighborsClassifier

In [ ]:
# Instantiate KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit the classifier
clf.fit(scaled_data_train, y_train)

# Predict on the test set
test_preds = clf.predict(scaled_data_test)

Evaluate Model

In [ ]:
# Complete the function
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds)))
    print("Recall Score: {}".format(recall_score(labels, preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)))
    
print_metrics(y_test, test_preds)

Improve model performance

In [ ]:
def find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    best_k = 0
    best_score = 0.0
    for k in range(min_k, max_k+1, 2):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_test)
        f1 = f1_score(y_test, preds)
        if f1 > best_score:
            best_k = k
            best_score = f1
    
    print("Best Value for k: {}".format(best_k))
    print("F1-Score: {}".format(best_score))

In [ ]:
find_best_k(scaled_data_train, y_train, scaled_data_test, y_test)